# 数据探索之假设检验


### 单个样本均值的假设检验



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import brfss
from scipy import stats  # 导入scipy的统计模块

%config InlineBackend.figure_format = 'retina'

df = brfss.ReadBrfss()  # 读取数据

In [2]:
df2 = df[['bmi', 'income']].dropna()
bmi = df2.bmi
bmi_rich = df2[df2.income == 8].bmi  # 富人bmi数据
bmi_ord = df2[df2.income != 8].bmi  # 普通人bmi数据

In [3]:
mu = np.mean(bmi)  # 计算bmi均值
mu

28.18812531332513

In [4]:
t_stats, p_value = stats.ttest_1samp(bmi_rich, mu)
print("p value is %.10f" % (p_value/2))  # 除以2是因为我们使用的是单边检验

p value is 0.0000000000


In [5]:
t_stats, p_value = stats.ttest_1samp(bmi_rich[:500], mu)  # 选择前500个数据进行检验
print("p value is %.10f" % (p_value/2))

p value is 0.2906513576


In [6]:
t_stats, p_value = stats.ttest_1samp(bmi_rich[:4000], mu)  # 选择前4000个数据进行检验
print("p value is %.10f" % (p_value/2))

p value is 0.0000660545



#### 蒙特卡洛模拟之 bootstrap 方法



In [7]:
def bootstrap_replicate_1d(data, func):   # 进行一次重新抽样，并返回检验统计量
    return func(np.random.choice(data, size=len(data)))  

def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)  # 初始一个空数组
    for i in range(size):   # 进行多次重新抽样
        bs_replicates[i] = bootstrap_replicate_1d(data, func)  
    return bs_replicates  # 返回多次抽样的检验统计量数组


def bootstrap_pvalue_1samp(data, pop_stats, func, size=1):
    sample_stats = func(data)  # 计算原有样本的检验统计量    
    translated_data = data - sample_stats + pop_stats  # 数据平移
    bs_replicates = draw_bs_reps(translated_data, func, size) # 重新抽样
    p = np.sum( bs_replicates < sample_stats) / size # 计算抽样统计量小于原有统计量的概率
    return p

In [8]:
bootstrap_pvalue_1samp(bmi_rich, mu, np.mean, size=10000)

0.0

In [9]:
bootstrap_pvalue_1samp(bmi_rich[:500], mu, np.mean, size=10000)

0.71260000000000001

In [11]:
bootstrap_pvalue_1samp(bmi_rich[:4000], mu, np.mean, size=10000)

0.0001

### 两个独立样本均值的假设检验



In [12]:
t_stats, p_value = stats.ttest_ind(bmi_rich, bmi_ord)
print("p value is %.10f" % (p_value/2))

p value is 0.0000000000


In [13]:
t_stats, p_value = stats.ttest_ind(bmi_rich[:500], bmi_ord[:500])
print("p value is %.10f" % (p_value/2))

p value is 0.1219871502


In [14]:
t_stats, p_value = stats.ttest_ind(bmi_rich[:1000], bmi_ord[:1000])
print("p value is %.10f" % (p_value/2))

p value is 0.0000846839


#### 蒙特卡洛模拟之Permutation方法



In [15]:
def diff_of_means(data_1, data_2):  
    diff = np.mean(data_2) - np.mean(data_1)  # 计算两组数据均值的差
    return diff


def permutation_sample(data1, data2):  # 产生新的分组数据
    data = np.concatenate((data1, data2))  # 合并两组数据
    permuted_data = np.random.permutation(data)  # 对合并后的数据进行重新排列
    perm_sample_1 = permuted_data[:len(data1)]  # 分成新的两组数据
    perm_sample_2 = permuted_data[len(data1):]
    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):  # 进行多次重新分组的操作
    perm_replicates = np.empty(size)
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates


def permutation_pvalue(data_1, data_2, func, size=1):  # 计算P值
    empirical_test_stats = func(data_1, data_2)
    perm_replicates = draw_perm_reps(data_1, data_2, func, size)
    p = np.sum(perm_replicates > empirical_test_stats) / len(perm_replicates)
    return p



In [16]:
permutation_pvalue(bmi_rich, bmi_ord, diff_of_means, size=10000)

0.0

In [17]:
permutation_pvalue(bmi_rich[:500], bmi_ord[:500], diff_of_means, size=10000)

0.1142

In [18]:
permutation_pvalue(bmi_rich[:1000], bmi_ord[:1000], diff_of_means, size=10000)

0.0001